<a href="https://colab.research.google.com/github/Hydrodynamical/apps/blob/main/HardSphereDynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from typing import List

# Define an error for post-collisional conditions
class SimulationError(Exception):
    def __init__(self, message, extra_data=None):
        super().__init__(message)
        self.extra_data = extra_data

    def __str__(self):
        return f'{super().__str__()} (extra data: {self.extra_data})'


In [2]:
class Particle:
  def __init__(self, mass, position, velocity, index):
    self.mass = mass
    self.position = position
    self.velocity = velocity
    self.index = index # index should be mathematicians index from 1, ...

  def __repr__(self):
    return f"--------------------\nmass = {self.mass}\nposition = {self.position}\nvelocity = {self.velocity}\nindex = {self.index}\n--------------------"

  def print(self):
    print(repr(self))

  @classmethod
  def from_existing(cls, existing_particle, new_position = None, new_velocity=None, new_index=None):
      # If no information is included, assign based on existing particle
      if new_position is None:
          new_position = existing_particle.position
      if new_velocity is None:
          new_velocity = existing_particle.velocity
      if new_index is None:
          new_index = existing_particle.index
      # Create a new Particle object with properties copied from the existing particle
      return cls(new_position, new_velocity, new_index)


In [3]:
# Define a total particle number for testing
N = 2

# Prepare masses, positions, and velocities
mass1 = np.array([1])
position1 = np.array([0, 0, 0])
velocity1 = np.array([1, 1, 1])

mass2 = np.array([2])
position2 = np.array([1, 2, 3])
velocity2 = np.array([2, 2, 2])

# Create a list to hold Particle objects
particles = []

# Create some Particle objects and add them to the list
particles.append(Particle(mass=mass1, position=position1, velocity=velocity1, index = 1))
particles.append(Particle(mass=mass2, position=position2, velocity=velocity2, index = 2))

# Loop through the list and print the attributes of each Particle object
for particle in particles:
    particle.print()

--------------------
mass = [1]
position = [0 0 0]
velocity = [1 1 1]
index = 1
--------------------
--------------------
mass = [2]
position = [1 2 3]
velocity = [2 2 2]
index = 2
--------------------


In [4]:
# Define a function which returns True if particles below a collision threshold
def isCollision(i: Particle, j: Particle, collisionThreshold) -> bool:
  relativePositionNorm = np.linalg.norm(i.position - j.position)
  if relativePositionNorm <= collisionThreshold:
    return True
  else:
    return False

# Check behavior
relativePositionNorm = np.linalg.norm(particles[0].position - particles[1].position)
print(relativePositionNorm) # output is 3.7416573867739413
print(isCollision(particles[0], particles[1], collisionThreshold=3))
print(isCollision(particles[0], particles[1], collisionThreshold=4))

3.7416573867739413
False
True


In [5]:
# Define a post-collisional velocity pair
def postCollisionalVelocities(i: Particle, j: Particle, collisionThreshold):
  # Calculate if particles are actually in collision
  relativePosition = i.position - j.position
  relativePositionNorm = np.linalg.norm(relativePosition)
  if relativePositionNorm > collisionThreshold:
    raise SimulationError("The particles are not colliding.", extra_data = relativePositionNorm)
  else:
    # Calculate unit relative position
    unitRelativePosition = (1/relativePositionNorm) * relativePosition

    # Calculate post collisional velocities
    normalizedMass_i = 2 * (j.mass/(i.mass + j.mass))
    normalizedMass_j = 2 * (i.mass/(i.mass + j.mass))
    postVelocity_i = i.velocity - normalizedMass_i * np.dot(i.velocity - j.velocity, unitRelativePosition) * unitRelativePosition
    postVelocity_j = j.velocity + normalizedMass_j * np.dot(i.velocity - j.velocity, unitRelativePosition) * unitRelativePosition

    # Create new post collisional particles
    i.velocity = postVelocity_i
    j.velocity = postVelocity_j


# This throws the SimulationError if uncommented
#postCollisionalVelocities(particles[0], particles[1], collisionThreshold = 3)

# Check behavior
print(particles[0])
print(particles[1])

postCollisionalVelocities(particles[0], particles[1], collisionThreshold=4)

print(particles[0])
print(particles[1])



--------------------
mass = [1]
position = [0 0 0]
velocity = [1 1 1]
index = 1
--------------------
--------------------
mass = [2]
position = [1 2 3]
velocity = [2 2 2]
index = 2
--------------------
--------------------
mass = [1]
position = [0 0 0]
velocity = [1.57142857 2.14285714 2.71428571]
index = 1
--------------------
--------------------
mass = [2]
position = [1 2 3]
velocity = [1.71428571 1.42857143 1.14285714]
index = 2
--------------------


In [6]:
from traitlets.traitlets import Float
# Here we define the linear dynamics
def linearFlow(gas: List[Particle], time: Float) -> List[Particle]:
  for particle in gas:
    particle.position = particle.position + time * particle.velocity

# Check behavior
print(particles[0])
print(particles[1])
linearFlow(particles, time = 10)
print(particles[0])
print(particles[1])

--------------------
mass = [1]
position = [0 0 0]
velocity = [1.57142857 2.14285714 2.71428571]
index = 1
--------------------
--------------------
mass = [2]
position = [1 2 3]
velocity = [1.71428571 1.42857143 1.14285714]
index = 2
--------------------
--------------------
mass = [1]
position = [15.71428571 21.42857143 27.14285714]
velocity = [1.57142857 2.14285714 2.71428571]
index = 1
--------------------
--------------------
mass = [2]
position = [18.14285714 16.28571429 14.42857143]
velocity = [1.71428571 1.42857143 1.14285714]
index = 2
--------------------
